# __Prepare raw data files__

### Setup

In [ ]:
%load_ext watermark
%load_ext autoreload

In [ ]:
%autoreload 2
%watermark -v -n -m -p numpy

In [ ]:
import sys
sys.path.append('../')
from pathlib import Path
import settings
import mne

## __Try pickle compression__

In [ ]:
import compress_pickle
import pickle

In [ ]:
SAMPLE = settings.DatasetConfig.SAMPLES_DIR / 'seizure_1.edf'    

In [ ]:
raw = mne.io.read_raw_edf(str(SAMPLE), preload=True, exclude=["BURSTS",
                                                            "ECG EKG-REF",
                                                            "EMG-REF",
                                                            "IBI",
                                                            "PHOTIC-REF",
                                                            "PULSE RATE",
                                                            "RESP ABDOMEN-RE",
                                                            "SUPPR"])

In [ ]:
raw.info

In [ ]:
type(raw.info['meas_date'])

Mongo DB

In [ ]:
from pymongo import MongoClient
import bson
import gridfs
import gzip

client = MongoClient()
db = client['eeg_assistant']

edf_raw = gridfs.GridFS(db, collection='edf_raw')

In [ ]:
with SAMPLE.open(mode='rb') as f:
    raw_bytes = f.read()
    print(type(raw_bytes))
    
    # serialization
    file_id = edf_raw.put(gzip.compress(raw_bytes))
        
    print(file_id)
    
    raw_data = edf_raw.get(file_id).read()
    print(type(raw_data))
    raw_data = gzip.decompress(raw_data)
    
    with NamedTemporaryFile(suffix='.edf') as tf:
        tf.write(raw_data)
        edf_file = mne.io.read_raw_edf(str(Path(tf.name)), preload=True, exclude=["BURSTS",
                                                                                "ECG EKG-REF",
                                                                                "EMG-REF",
                                                                                "IBI",
                                                                                "PHOTIC-REF",
                                                                                "PULSE RATE",
                                                                                "RESP ABDOMEN-RE",
                                                                                "SUPPR"])

In [ ]:
edf_file.info

## __Samples__

In [ ]:
SAMPLE = settings.DatasetConfig.SAMPLES_DIR / 'abnormal_1.edf'

SAMPLE

In [ ]:
raw = mne.io.read_raw_edf(str(SAMPLE), preload=True, exclude=["BURSTS",
                                                            "ECG EKG-REF",
                                                            "EMG-REF",
                                                            "IBI",
                                                            "PHOTIC-REF",
                                                            "PULSE RATE",
                                                            "RESP ABDOMEN-RE",
                                                            "SUPPR"])

In [ ]:
raw.close()

In [ ]:
raw.info

In [ ]:
raw.ch_names

In [ ]:
raw.n_times

In [ ]:
(raw.n_times / raw.info['sfreq']) / 60

In [ ]:
#raw_cropped = raw.crop(tmin=100)

In [ ]:
raw.plot()